<h1> Where to open an ice cream shop in Salvador ? </h1>


<h2>1. Import required libraries </h2>

In [1]:
# Import required libraries

import pandas as pd
import numpy as np
from bs4 import BeautifulSoup 
import requests
from html.parser import HTMLParser
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    altair-3.2.0               |           py36_0         770 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.3 MB

The following NEW packages will be 

<h2> 2. Set parameters required when using Foursquare API </h2>

For this work, we are going to use FourSquare API. 
Latitute and longitude are required parameters for using Foursquare API

In [4]:
latitude= -12.9704 # Latitude for Salvador
longitude= -38.5124  # Longitude for Salvador
CLIENT_ID = '4AQNRD4F2JQIMHF1FZ5QXH1LAS3NC2ZHNOMEGJSXG3CY4B3J' #  Foursquare ID
CLIENT_SECRET = 'KDJVVN0OQO3CF5VRHYIHOEYORIW2ULLKUDQ5UQR5YSITYBZ3' #  Foursquare Secret
VERSION = '20180605' # Foursquare API version


<h2> 3. Search for beaches in Salvador </h2>

Using Foursquare, get a json file containing venue data for beaches in Salvador

In [8]:

radius = 15000 # define radius
LIMIT = 200

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&query={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT,
    'beach')

results = requests.get(url).json()


<h2> 4. Create a dataframe with venues data, from Foursquare json </h2>

This section will convert the json file returned by Foursquare API into a dataframe with the following columns: id, name, categories, lat and lng.
<li> id is the unique foursquare id for the beach
<li> name  is the name for the beach
<li> categories can be initially filled with Beach, Harbor, Restaurant, etc... later we will clean the data to make drop the rows that are not beaches
<li> lat is the beach latitude
<li> lng is the beach longitude

In [12]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [13]:
venues = results['response']['groups'][0]['items']
    
salvador_beaches = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.id','venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
salvador_beaches = salvador_beaches.loc[:, filtered_columns]

# filter the category for each row
salvador_beaches['venue.categories'] = salvador_beaches.apply(get_category_type, axis=1)

# clean columns
salvador_beaches.columns = [col.split(".")[-1] for col in salvador_beaches.columns]

salvador_beaches.head()

,id,name,categories,lat,lng
0,4bce20f1ef109521f18c8386,Praia do Porto da Barra,Beach,-13.003226,-38.532317
1,4f50b31de4b0a3428a690578,Praia da Paciência,Beach,-13.010340,-38.499790
2,4ba4b403f964a52099ad38e3,Porto de Salvador,Harbor / Marina,-12.961060,-38.506628
3,4cc4668ef49676b0188e72d5,Praia do Buracão,Beach,-13.013216,-38.483506
4,520755f893cdb1bf12924f72,Praia da Barra,Beach,-13.010119,-38.531048


In [14]:
# Clean the data frame to only keep rows where category is equals to beach

indexNames = salvador_beaches[(salvador_beaches['categories'] != 'Beach')].index # Remove rows that are not Beach
salvador_beaches.drop(indexNames , inplace=True)
salvador_beaches = salvador_beaches[salvador_beaches.name.str.contains("Praia")] # Keep only rows that have Praia as part of the name. Praia means Beach
salvador_beaches

,id,name,categories,lat,lng
0,4bce20f1ef109521f18c8386,Praia do Porto da Barra,Beach,-13.003226,-38.532317
1,4f50b31de4b0a3428a690578,Praia da Paciência,Beach,-13.010340,-38.499790
3,4cc4668ef49676b0188e72d5,Praia do Buracão,Beach,-13.013216,-38.483506
4,520755f893cdb1bf12924f72,Praia da Barra,Beach,-13.010119,-38.531048
5,4d97688597d06ea878331b0b,Praia de Ondina,Beach,-13.009268,-38.511682
6,4efb21a4d5fba46862a67295,Praia de Boa Viagem,Beach,-12.930216,-38.509697
7,4dba3ca66a23e2622d9d2ddd,Praia do Rio Vermelho,Beach,-13.011269,-38.492438
9,507f15b7e4b08ec3d1a29645,Praia da Ribeira,Beach,-12.931015,-38.491591
11,5234947b11d2a87016d720e1,Praia Madre de Deus,Beach,-12.933546,-38.487054
13,4c221fde9390c9b6da01cacd,Praia Jardim de Alah,Beach,-12.994726,-38.442606


<h2> 5. Plot Salvador map with located beaches </h2>

In [16]:
# create map of Salvador using latitude and longitude values
map_ssa = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, name in zip(salvador_beaches['lat'], salvador_beaches['lng'], salvador_beaches['name']):
#    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(name, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,

        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_ssa)  
    
map_ssa


<h2> 6. Get ice cream shops close to up 300 meters from the beach </h2>

For each beach, locate existing ice cream shops that are up to 300 metes.
Return at most 5 ice cream shops per beach.
Create a list of beaches that don't have close ice cream shops

In [17]:
beaches_without_ice_cream_shops = []


for row in salvador_beaches.iterrows():
    ice_cream_url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&query={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    row[1][3], 
    row[1][4], 
    300, 
    5,
    'ice cream')

    results_ice_cream = requests.get(ice_cream_url).json()
    venues = results_ice_cream['response']['groups'][0]['items']
    ice_cream_shops = json_normalize(venues) # flatten JSON
    try:
        filtered_columns = ['venue.id','venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
        ice_cream_shops = ice_cream_shops.loc[:, filtered_columns]
        # filter the category for each row
        ice_cream_shops['venue.categories'] = ice_cream_shops.apply(get_category_type, axis=1)
        # clean columns
        ice_cream_shops.columns = [col.split(".")[-1] for col in ice_cream_shops.columns]
        print('Ice cream shops close to '+row[1][1])
        print(ice_cream_shops)
    except:
        print('No ice cream shops close to '+row[1][1])
        beaches_without_ice_cream_shops.append(row[1][1])
        
ice_cream_shops.head()

Ice cream shops close to Praia do Porto da Barra
                         id                 name      categories        lat  \
0  5032634be4b060c1a6d60897        Gelatto Bahia  Ice Cream Shop -13.002674   
1  54daa4b7498e925b41d80ded  Sorveteria da Bahia  Ice Cream Shop -13.002973   
2  5a4d7a215f68b95208e9c2ba          Doce Gelato  Ice Cream Shop -13.004928   
3  4e78fe6881dca7a1abe1fa0b             Yofrozen  Ice Cream Shop -13.004913   

         lng  
0 -38.532132  
1 -38.532019  
2 -38.533014  
3 -38.531220  
No ice cream shops close to Praia da Paciência
Ice cream shops close to Praia do Buracão
                         id                  name      categories        lat  \
0  4dc6f59252b1e8f9f7f9f93c  Sorveteria Primavera  Ice Cream Shop -13.013261   

         lng  
0 -38.486007  
Ice cream shops close to Praia da Barra
                         id                name      categories        lat  \
0  58a76e502321c67cca3a8879  Chiquinho Sorvetes  Ice Cream Shop -13.009170   
1  5

""


<h2> 7.Results </h2>

This section display the list of beaches without a close ice cream shop.
The list was build in last section

In [18]:
beaches_without_ice_cream_shops

['Praia da Paciência',
 'Praia de Ondina',
 'Praia Madre de Deus',
 'Praia Jardim de Alah',
 'Praia de Armação',
 'Praia de Amaralina',
 'Praia de Mar Grande',
 'Praia da Pituba',
 'Praia de Patamares',
 'Praia de Piatã',
 'Praia Da Armacao',
 'Praia de Pituaçu',
 'Praia dos Buzios',
 'Praia De Periperi',
 'Praia Tubarão',
 'Praia de Piatã',
 'Praia de Piatã',
 'Praia De Piatã',
 'Praia Grande',
 'Praia da 3ª Ponte',
 'Praia Farol da Barra',
 'Praia de Placafor',
 'Praia do Alvejado',
 'Praia Do Jaburu',
 'Praia do Bom Despacho',
 'Praia do Flamengo - Salvador',
 'Praia da Penha']